Based on content from
https://towardsdatascience.com/image-classification-python-keras-tutorial-kaggle-challenge-45a6332a58b8

In [1]:
from PIL import Image
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers. normalization import BatchNormalization

Using TensorFlow backend.


In [2]:
import numpy as np
import os
import imageio
import matplotlib.pyplot as plt
from random import shuffle

In [37]:
naming_dict = {} # id: breed
f = open("labels.csv", "r")
fileContents = f.read()
fileContents = fileContents.split('\n')
for i in range(len(fileContents) - 1):
    fileContents[i] = fileContents[i].split(',')
    naming_dict[fileContents[i][0]] = fileContents[i][1]

In [4]:
breeds = naming_dict.values()
breed_set = set(breeds)
counting_dict = {}
for i in breed_set:
    counting_dict[i] = 0

In [42]:
def create_dir(name):
    if not os.path.isdir(name):
        os.mkdir(name)
create_dir('labeled_train')

In [25]:
for img in os.listdir('./train'):
    imgName = img.split('.')[0]
    label = naming_dict[str(imgName)]
    counting_dict[label] += 1
    path = os.path.join('./train', img)
    saveName = f"./labeled_train/{label}-{str(counting_dict[label])}.jpg"
    image_data = np.array(Image.open(path))
    imageio.imwrite(saveName, image_data)

In [3]:
def label_img(name):
    word_label = name.split('-')[0]
    if word_label == 'golden_retriever' : return np.array([1, 0])
    if word_label == 'shetland_sheepdog': return np.array([0, 1])

In [4]:
def get_size_stats(DIR):
    heights = []
    widths = []
    for img in os.listdir(DIR):
        path = os.path.join(DIR, img)
        data = np.array(Image.open(path))
        heights.append(data.shape[0])
        widths.append(data.shape[1])
    avg_height = sum(heights) / len(heights)
    avg_width = sum(widths) / len(widths)
    
    print("Average Height: " + str(avg_height))
    print("Max Height: " + str(max(heights)))
    print("Min Height: " + str(min(heights)))
    print('\n')
    print("Average Width: " + str(avg_width))
    print("Max Width: " + str(max(widths)))
    print("Min Width: " + str(min(widths)))

In [28]:
get_size_stats('labeled_train')

Average Height: 386.74721189591077
Max Height: 2562
Min Height: 102


Average Width: 443.33153981608297
Max Width: 3264
Min Width: 97


In [5]:
IMG_SIZE = 300

In [22]:
def load_training_data(DIR='labeled_train'):
    train_data = []
    for img in os.listdir(DIR):
        label = label_img(img)
        if label is None:
            continue
        path = os.path.join(DIR, img)
        img = Image.open(path)
        img = img.convert('L')
        img = img.resize((IMG_SIZE, IMG_SIZE), Image.ANTIALIAS)
        train_data.append([np.array(img), label])
        
        # Basic Data Augmentation - Horizontal Flipping
        flip_img = Image.open(path)
        flip_img = flip_img.convert('L') # L means grayscale
        flip_img = flip_img.resize((IMG_SIZE, IMG_SIZE), Image.ANTIALIAS)
        flip_img = np.array(flip_img)
        flip_img = np.fliplr(flip_img)
        train_data.append([flip_img, label])
        
    shuffle(train_data)
    return train_data

In [23]:
train_data = load_training_data()

In [24]:
plt.imshow(train_data[43][0], cmap = 'gist_gray')

In [44]:
# create cross validation set
cv_data = train_data[-100:]
train_data = train_data[:100]

In [45]:
model = Sequential()
model.add(Conv2D(32, kernel_size = (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(96, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(32, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
#model.add(Dropout(0.3))
model.add(Dense(2, activation = 'softmax'))

In [51]:
trainImages = np.array([i[0] for i in train_data]).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
trainLabels = np.array([i[1] for i in train_data])
cvImages = np.array([i[0] for i in cv_data]).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
cvLabels = np.array([i[1] for i in cv_data])

In [47]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [48]:
model.fit(trainImages, trainLabels, batch_size =  50, epochs = 5, verbose = 1)

Epoch 1/5
100/100 [==============================] - 8s 76ms/step - loss: 1.2038 - acc: 0.4900
Epoch 2/5
100/100 [==============================] - 6s 56ms/step - loss: 0.2622 - acc: 0.9200
Epoch 3/5
100/100 [==============================] - 5s 55ms/step - loss: 0.1286 - acc: 0.9700
Epoch 4/5
100/100 [==============================] - 6s 56ms/step - loss: 0.0671 - acc: 0.9800
Epoch 5/5
100/100 [==============================] - 6s 55ms/step - loss: 0.0547 - acc: 0.9900


In [52]:
loss, acc = model.evaluate(cvImages, cvLabels, verbose = 0)
print(acc * 100)

63.0


In [53]:
len(os.listdir('train'))

10222

In [55]:
trainImages.shape

(100, 300, 300, 1)